숫자 인식기를 한 단계 더 발전시켜 "Number One"과 같이 단어 내에서 숫자를 감지하고 싶다고 가정해 보겠습니다. 기본 spaCy토큰 속성을 활용하거나 자체 논리를 작성하여 이러한 엔터티를 감지할 수 있습니다.

메모:

EntityRecognizer이 예제에서는 Presidio의 기본 인식기 를 구현하는 새 클래스를 만듭니다 . 이 추상 클래스를 사용하려면 load메서드와 analyze메서드를 구현해야 합니다.

각 인식기는 NlpArtifacts입력 텍스트에 대해 미리 계산된 속성을 보유하는 type 의 개체를 허용합니다.

새 인식기는 다음 구조를 가져야 합니다.

In [1]:
from typing import List
from presidio_analyzer import EntityRecognizer, RecognizerResult
from presidio_analyzer.nlp_engine import NlpArtifacts


class MyRecognizer(EntityRecognizer):
    def load(self) -> None:
        """No loading is required."""
        pass

    def analyze(
        self, text: str, entities: List[str], nlp_artifacts: NlpArtifacts
    ) -> List[RecognizerResult]:
        """
        Logic for detecting a specific PII
        """
        pass

예를 들어 숫자 또는 알파벳(예: 45) 형식의 숫자 감지:

In [2]:
from typing import List
from presidio_analyzer import EntityRecognizer, RecognizerResult
from presidio_analyzer.nlp_engine import NlpArtifacts


class NumbersRecognizer(EntityRecognizer):

    expected_confidence_level = 0.7  # expected confidence level for this recognizer

    def load(self) -> None:
        """No loading is required."""
        pass

    def analyze(
        self, text: str, entities: List[str], nlp_artifacts: NlpArtifacts
    ) -> List[RecognizerResult]:
        """
        Analyzes test to find tokens which represent numbers (either 123 or One Two Three).
        """
        results = []

        # iterate over the spaCy tokens, and call `token.like_num`
        for token in nlp_artifacts.tokens:
            if token.like_num:
                result = RecognizerResult(
                    entity_type="NUMBER",
                    start=token.idx,
                    end=token.idx + len(token),
                    score=self.expected_confidence_level,
                )
                results.append(result)
        return results


# Instantiate the new NumbersRecognizer:
new_numbers_recognizer = NumbersRecognizer(supported_entities=["NUMBER"])


이 인식기는 를 필요로 하므로 흐름 NlpArtifacts의 일부로 호출해야 합니다 .AnalyzerEngine

In [3]:
from presidio_analyzer import AnalyzerEngine

text3 = "Roberto lives in Five 10 Broad st."
analyzer = AnalyzerEngine()
analyzer.registry.add_recognizer(new_numbers_recognizer)

numbers_results2 = analyzer.analyze(text=text3, language="en")
print("Results:")
print("\n".join([str(res) for res in numbers_results2]))

Results:
type: PERSON, start: 0, end: 7, score: 0.85
type: NUMBER, start: 17, end: 21, score: 0.7
type: NUMBER, start: 22, end: 24, score: 0.7


분석기는 다른 인식기(이 경우 PERSON)에서 다른 유형의 PII 엔터티를 포함하여 숫자 및 알파벳 숫자를 모두 선택할 수 있었습니다.